# Project 5 Loading classifier

# Step 1

Libraries

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score

# Step 2

Loading classifier

In [2]:
np.random.seed(123)
torch.manual_seed(123)

In [3]:
class torch_classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.dense0 = nn.Linear(30,8)
    torch.nn.init.normal_(self.dense0.weight, mean=0.0, std=0.05)

    self.dense1 = nn.Linear(8,8)
    torch.nn.init.normal_(self.dense1.weight, mean=0.0, std=0.05)

    self.dense2 = nn.Linear(8,1)
    self.activation = nn.ReLU()

    self.dropout = nn.Dropout(0,2)
    self.output = nn.Sigmoid()

  def forward(self, X):
    X = self.dense0(X)
    X = self.activation(X)
    X = self.dropout(X)
    X = self.dense1(X)
    X = self.activation(X)
    X = self.dropout(X)
    X = self.dense2(X)
    X = self.output(X)
    return X

In [13]:
classifier = torch_classifier()

In [14]:
state_dict = torch.load('/content/brest_cancer.pth')

In [16]:
state_dict()

OrderedDict([('dense0.weight',
              tensor([[ 1.6502e-01,  6.0314e-02,  5.0601e-01,  1.4060e-01, -3.8562e-02,
                       -1.3699e-01, -6.6027e-04,  1.2158e-01, -1.7807e-03,  1.7545e-01,
                        5.6647e-02,  5.3652e-02, -9.2483e-03, -1.1478e-02, -3.3487e-02,
                        5.3917e-02,  2.8183e-01,  3.2757e-01,  1.4797e-01, -4.9551e-01,
                        8.6561e-02, -1.5248e-01,  3.2011e-01, -1.6158e-01, -1.2600e-03,
                       -3.8763e-02, -7.6130e-02,  8.3725e-02, -4.2798e-02, -4.4904e-01],
                      [ 6.1736e-02, -2.0170e-02,  3.4046e-01,  1.3590e-02, -1.6125e-01,
                        6.8624e-03, -3.3806e-02,  2.3744e-01,  3.9160e-02, -2.0839e-01,
                       -8.8423e-02,  1.4717e-02,  3.5499e-02, -1.2219e-01, -1.6580e-02,
                        3.2146e-01,  2.0999e-01,  2.5235e-01, -3.7452e-01, -4.2356e-01,
                        1.3984e-01, -6.2029e-02,  3.1465e-01, -8.8413e-02, -7.3453e-02,


In [18]:
classifier.load_state_dict(state_dict())

<All keys matched successfully>

In [19]:
new = torch.tensor([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]], dtype = torch.float)

In [20]:
classifier.eval()

torch_classifier(
  (dense0): Linear(in_features=30, out_features=8, bias=True)
  (dense1): Linear(in_features=8, out_features=8, bias=True)
  (dense2): Linear(in_features=8, out_features=1, bias=True)
  (activation): ReLU()
  (dropout): Dropout(p=0, inplace=2)
  (output): Sigmoid()
)

In [22]:
forecasting = classifier(new)
forecasting = (forecasting.detach().numpy() > 0.5)

In [23]:
forecasting

array([[ True]])

In [24]:
forecasters = pd.read_csv('/content/entradas_breast.csv')
labels = pd.read_csv('/content/saidas_breast.csv')

In [26]:
forecasters = torch.tensor(np.array(forecasters), dtype= torch.float)
labels = torch.tensor(np.array(labels), dtype= torch.float)

In [28]:
forecast = classifier.forward(forecasters).detach()

In [29]:
forecast

tensor([[1.5287e-13],
        [6.8996e-09],
        [3.7021e-09],
        [2.4409e-07],
        [2.4401e-05],
        [2.6885e-02],
        [5.7855e-06],
        [1.5588e-02],
        [6.4386e-06],
        [5.4512e-20],
        [5.1804e-03],
        [6.4926e-07],
        [4.1891e-01],
        [6.6038e-01],
        [9.3073e-01],
        [8.2656e-02],
        [2.0035e-05],
        [1.9676e-07],
        [3.5463e-13],
        [9.3862e-01],
        [7.0625e-01],
        [1.0000e+00],
        [4.6753e-02],
        [3.3682e-15],
        [6.9563e-12],
        [9.5128e-09],
        [8.9953e-02],
        [3.6340e-04],
        [1.4009e-06],
        [2.8140e-04],
        [2.7844e-09],
        [6.1576e-04],
        [8.3040e-06],
        [5.1428e-07],
        [1.8882e-06],
        [4.3848e-05],
        [3.0962e-01],
        [9.9999e-01],
        [1.4378e-02],
        [1.5542e-05],
        [8.9272e-02],
        [9.7002e-01],
        [6.4094e-03],
        [8.0817e-03],
        [4.0408e-01],
        [3

In [31]:
F.binary_cross_entropy(forecast, labels).numpy() # error

array(0.14421439, dtype=float32)

In [32]:
accuracy_score(labels.numpy(), (forecast > 0.5).numpy())

0.9490333919156415